## Mini OSS Insight Workshop

👏 欢迎来到 Mini OSS Insight 的 Workshop!

这是关于 Workshop 的完整指南，你可以通过阅读该指南一步一步地搭建起一个 Mini 版的 OSS Insight，并在过程掌握一些使用 TiDB / TiDB Cloud 进行数据分析的技巧。

### 准备

在开始之前，你需要确保在你的开发环境中已经安装了以下软件/工具：

- 可供连接的 TiDB 集群 - 用于存储数据的数据库
- MyCLI - 连接到 TiDB 集群
- Node.js 18.x and above - API Server 的运行时环境
- PNPM - Node.js 的包管理器
- Python 3.x and pip - Jupyter Notebook 需要的运行时环境

#### 1. 启动 TiDB 集群

首先, 你需要开启一个 TiDB 集群。你可以通过查看文档 [TiDB Cloud Quick Start](https://docs.pingcap.com/tidbcloud/tidb-cloud-quickstart#step-1-create-a-tidb-cluster) 学习如何创建一个 **Serverless tier** 集群，这可能会需要 20～25s 的时间。

在你创建完集群，你可以进入到集群的详情页。在这个页面，你可以在 **Connection** 面板中找到数据库集群的连接信息。

<center>
  <img align="middle" width="800" alt="Serverless Tier Cluster Manage Interface" src="https://user-images.githubusercontent.com/5086433/204476069-0ddbdf6f-419c-4291-b929-ccfbd2f5ea5f.png">
  <p><i>Serverless Tier 集群管理界面</i></p>
</center>

You can enter to the cluster's security settings window by clicking on the modify menu in the upper right corner of the cluster details page and generate a root user password.

<center>
  <img width="480" alt="The Cluster Modify Menu" src="https://user-images.githubusercontent.com/85985765/204876779-3a4c6ac4-8814-47cd-b82a-40eb5e4d8f96.png">
  <p><i>The Cluster Modify Menu</i></p>
</center>

<center>
  <img width="720" alt="Security Settings" src="https://user-images.githubusercontent.com/85985765/204877348-5c3e9012-f7bf-42e9-8a03-fd9f14bfc826.png">
  <p><i>Security Settings</i></p>
</center>

#### 3. 个人 GitHub 的 Access Token

You need to prepare a personal access token to allow the application to access the data through the GitHub API.

> **Note**
>
> If you are viewing this document from GitHub codespace, you can skip this step because `GITHUB_TOKEN` is set by default in the codespace environment.
>  

You can learn how to generate one by reading: [Creating a personal access token](https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/creating-a-personal-access-token). Or just click this [link](https://github.com/settings/personal-access-tokens/new) to generate your personal access token quickly.

<center>
  <img align="middle" width="800" alt="Create a New GitHub Personal Access Token" src="https://user-images.githubusercontent.com/5086433/204564273-93cccbe4-d10a-4d1b-a9d1-112a1144712a.png">
  <p><i>Create a new GitHub personal access token</i></p>
</center>


#### 4. 设置环境变量


In [2]:
import getpass
import os

api_server_dot_env="./packages/api-server/.env"
if os.path.exists(api_server_dot_env): 
    con = input(api_server_dot_env + " file already exists, do you want to overwrite it? (y/n) [y]")
    if con != 'y':
        exit();

# Config GitHub personal access token.
github_token = ""
if os.getenv('GITHUB_TOKEN') is None:
    github_token = getpass.getpass(prompt='Enter your personal access token of GitHub: ')
else:
    github_token = os.getenv('GITHUB_TOKEN')

# Config database connection.
db_endpoint = input("The endpoint of TiDB cluster: ")
db_port = int(input("The port of TiDB cluster [4000]: ")) or 4000
db_username = input("The username of TiDB cluster: ")
db_password = getpass.getpass(prompt='Enter the password of TiDB cluster: ')
db_name = "ossinsight"
db_enable_ssl = input("Enable ssl connection to the TiDB cluster? (y/n) [y]") or "y"
db_ssl_config='&ssl={"minVersion":"TLSv1.2"}'
if db_enable_ssl == "n":
    db_ssl_config = ""

# Write to ./packages/api-server/.env file.
with open(api_server_dot_env, "w") as file:
    file.write(
        "DATABASE_URL=mysql://{}:{}@{}:{}/{}?connectionLimit=100&queueLimit=10000{}\n".format(
            db_username, db_password, db_endpoint, db_port, db_name, db_ssl_config
        )
    )
    file.write("ENABLE_CACHE=false\n")
    file.write("GITHUB_ACCESS_TOKENS={}\n".format(github_token))

print("Setup successfully!")

Setup successfully!


#### 5. 验证是否能够连接到 TiDB 集群

在终端上执行下面的 SQL 语句验证是否能够连接到 TiDB 集群

```bash
mycli -h ${DB_ENDPOINT} -P ${DB_PORT} -u ${DB_USERNAME} -p ${DB_PASSWORD} -D test \
    --ssl-ca=/etc/ssl/certs/ca-certificates.crt \
    --ssl-verify-server-cert \
     -e 'SELECT tidb_version()\G'
```

如果成功执行，你会看到 TiDB 集群的版本信息：

```
tidb_version()
Release Version: v6.3.0-serverless
Edition: Community
Git Commit Hash: e87c16b215d518aed4921b8ef3b13e90e3ed6e2d
Git Branch: release-6.3-serverless
UTC Build Time: 2022-11-25 09:31:28
GoVersion: go1.19
Race Enabled: false
TiKV Min Version: 6.1.0
Check Table Before Drop: false
Store: tikv
```

### 加载数据

GitHub 成立于 2008 年，在 2011 年二月份的时候，GitHub 提供了一个 `https://api.github.com/events` 接口来对外输出网站上的公开事件数据。

> **GitHub API 文档**
> 
> Link: https://docs.github.com/en/rest/activity/events
>


OSS Insight 已经收集了超过 53 亿条公开事件数据，你可以通过访问 API 或在网站上查看当前的事件总数：

In [11]:
!curl -s https://api.ossinsight.io/q/events-total | jq -r ".data[0]"

{
  "cnt": 5319456119,
  "latest_created_at": "2022-12-01T19:18:00.000Z",
  "latest_timestamp": 1669922280
}


#### 导入实时数据

通过 [GitHub 的 API 文档](https://docs.github.com/en/rest/activity/events), 我们知道可以通过请求 `/events` API 来获取准实时的 events 数据。你可以执行下面的命令查看这个接口返回数据的结构：

In [2]:
!curl -s \
  -H "Accept: application/vnd.github.v3+json" \
  -H "Authorization: token $GITHUB_TOKEN" \
  https://api.github.com/events

[
  {
    "id": "25599598187",
    "type": "PushEvent",
    "actor": {
      "id": 40635516,
      "login": "DaniBootCamp",
      "display_login": "DaniBootCamp",
      "gravatar_id": "",
      "url": "https://api.github.com/users/DaniBootCamp",
      "avatar_url": "https://avatars.githubusercontent.com/u/40635516?"
    },
    "repo": {
      "id": 573120852,
      "name": "carlosmavogaro/amoung-us-grupo-1",
      "url": "https://api.github.com/repos/carlosmavogaro/amoung-us-grupo-1"
    },
    "payload": {
      "push_id": 11852249371,
      "size": 1,
      "distinct_size": 1,
      "ref": "refs/heads/daniel",
      "head": "a6a49030886b478c7047d22e556694e4f39b5b05",
      "before": "880af9cd744c901d0737f0bf0d27a61393d1f9dc",
      "commits": [
        {
          "sha": "a6a49030886b478c7047d22e556694e4f39b5b05",
          "author": {
            "email": "titodani@gmail.com",
            "name": "DaniBootCamp"
          },
          "message": "buscando impostor",
          "distin

In [ ]:
%%!

docker-compose pull
docker-compose up web

#### 导入历史数据

由于通过 `/events` API 我们无法完全获取到所有的历史数据, 在 OSS Insight 的数据准备阶段，我们利用了 [GH Archive](https://gharchive.org) 上归档数据，将其导入数据库当中.

| Query | Downloadable Files |
| ---- | ---- |
| Activity for 1/1/2015 @ 3PM UTC |	`https://data.gharchive.org/2015-01-01-15.json.gz` |
| Activity for 1/1/2015 | `https://data.gharchive.org/2015-01-01-{0..23}.json.gz` |
| Activity for all of January 2015 | `https://data.gharchive.org/2015-01-{01..31}-{0..23}.json.gz` |

为了方便演示我们在 AWS S3 上保存了一份示例数据，你可以使用 TiDB Cloud 的 Import 功能将其导入数据库当中：

S3 URI:

```
s3://ossinsight/workshop/oss_database_sample/
```

Role ARN:

```
arn:aws:iam::494090988690:role/tidb-cloud-reader
```

## 一些背景知识

#### 优化器

SQL 是一种声明式语言，而非过程性的语言。也就是说，它描述的是最终结果应该如何，而非按顺序执行的步骤。因此在 TiDB 在处理 SQL 语句的过程中，除了要确保能够正确返回语句所描述的最终结果，还要确定 SQL 语句的执行过程，并且尽可能的对 SQL 语句进行性能优化，从而得到最佳的执行计划。

SQL 性能优化的过程，可以理解为 GPS 导航的过程。你提供地址后，GPS 软件利用各种统计信息（例如以前的行程、速度限制等元数据，以及实时交通信息）规划出一条最省时的路线。这与 TiDB 中的 SQL 性能优化过程相对应。

我们通常会把负责这部分 SQL 优化工作的组件成为**优化器（Optimizer）**。

如下图所示，一条 SQL 语句在经过语法解析和校验之后，经过优化器的逻辑优化和物理优化来制定最终的执行计划，交给执行器（Executor）去执行。和 GPS 软件一样，优化器在进行执行计划规划的时候也会利用到一些统计信息来进行评估。

<center>
  <img width="640" alt="The Cluster Modify Menu" src="https://user-images.githubusercontent.com/85985765/205136445-3d9dd676-56bc-4f75-bf6d-cdb1af8c1b79.png">
  <p><i>SQL 优化流程图</i></p>
</center>

上图其实是一个简化的示意图，因为它只描述了以 TiKV 为存储引擎的情况。但是在 TiDB 支持 HTAP 能力后，引进了负责分析负载的 TiFlash 存储引擎。这给优化器带来了新的挑战，优化器需要在制定执行计划和进行物理
优化时需要考虑：

> 这条 SQL 应该使用哪个存储引擎执行性能会更好，选 TiKV 还是 TiFlash？

#### Optimizer Hints

> 如果优化器难以抉择，那我们不妨给它一点提示，告诉它我们希望使用哪种存储引擎执行?

TiDB 的优化器提供了 `READ_FROM_STORAGE` Hint 来允许开发者在 SQL 语句级别来控制哪一张表应该使用哪个执行引擎去执行。例如，下在面这条 SQL 语句当中，TiDB 从 TiFlash 上读取 t1 表的数据，在 TiKV 上读取 t2 的数据。

```sql
SELECT /*+ READ_FROM_STORAGE(TIFLASH[t1], TIKV[t2]) */ t1.a FROM t t1, t t2 WHERE t1.a = t2.a;
```

#### 代价模型

> 使用 Optimizer Hints 很简单，但是有没有更智能的方法，特别是在做临时查询时，优化器能否智能的选择应该使用哪种引擎查询效率最高？

<center>
  <img width="640" alt="物理优化中的代码模型" src="https://user-images.githubusercontent.com/85985765/205192450-c23f2199-1ab1-41d2-90f3-c17c5d6e252e.png">
  <p><i>SQL 优化流程图</i></p>
</center>

TiDB v6.2.0 引入了新的代价模型 Cost Model Version 2。

Cost Model Version 2 对代价公式进行了更精确的回归校准，调整了部分代价公式，比此前版本的代价公式更加准确。

OSS Insight 使用了该版本的 Cost Model, 但是因为目前 Cost Model Version 2 还处于实验阶段，需要通过执行以下 SQL 语句进行启用：

In [ ]:
set global tidb_enable_new_cost_interface = 'ON';
set global tidb_cost_model_version = 2;

在后续版本当中，Cost Model Version 2 会替代掉原有的 Cost Model 作为默认的代码模型。

## Demo SQL



#### Example 1


In [ ]:
SELECT * FROM github_events WHERE actor_login = 'ti-chi-bot' LIMIT 1;

In [ ]:
EXPLAIN SELECT * FROM github_events WHERE actor_login = 'ti-chi-bot' LIMIT 1;

#### Example 2

In [ ]:
SELECT
  event_month, stars
FROM (
  SELECT
    date_format(created_at, '%Y-%m-01') as event_month,
    COUNT(1) OVER (ORDER BY date_format(created_at, '%Y-%m-01')) AS stars,
    ROW_NUMBER() OVER (PARTITION BY date_format(created_at, '%Y-%m-01')) AS row_num
  FROM
    github_events
  WHERE
    type = 'WatchEvent'
    AND repo_id = 41986369
  ORDER BY 1
) sub
WHERE row_num = 1
;

#### Example 3

In [ ]:
WITH bots_with_first_seen AS (
    SELECT
        actor_login, MIN(YEAR(created_at)) AS first_seen_at
    FROM github_events ge
    WHERE
        actor_login REGEXP '^(bot-.+|.+bot|.+\\[bot\\]|.+-bot-.+|robot-.+|.+-ci-.+|.+-ci|.+-testing|.+clabot.+|.+-gerrit|k8s-.+|.+-machine|.+-automation|github-.+|.+-github|.+-service|.+-builds|codecov-.+|.+teamcity.+|jenkins-.+|.+-jira-.+)$'
    GROUP BY actor_login
    ORDER BY first_seen_at
),  acc AS (
    SELECT
        COUNT(actor_login) OVER (ORDER BY first_seen_at) AS cnt,
        first_seen_at AS event_year
    FROM
        bots_with_first_seen AS bwfs
    ORDER BY event_year
)
SELECT ANY_VALUE(cnt) AS bots_total, event_year
FROM acc
GROUP BY event_year
ORDER BY event_year;

开启 TiFlash 副本

In [ ]:
ALTER TABLE gharchive_dev.github_events SET TIFLASH REPLICA 1;

In [ ]:
SELECT * FROM information_schema.tiflash_replica WHERE TABLE_SCHEMA = 'ossinsight' and TABLE_NAME = 'github_events';

再次执行 Example 3 中的查询语句：

In [ ]:
WITH bots_with_first_seen AS (
    SELECT
        actor_login, MIN(event_year) AS first_seen_at
    FROM github_events ge
    WHERE
        actor_login REGEXP '^(bot-.+|.+bot|.+\\[bot\\]|.+-bot-.+|robot-.+|.+-ci-.+|.+-ci|.+-testing|.+clabot.+|.+-gerrit|k8s-.+|.+-machine|.+-automation|github-.+|.+-github|.+-service|.+-builds|codecov-.+|.+teamcity.+|jenkins-.+|.+-jira-.+)$'
    GROUP BY actor_login
    ORDER BY first_seen_at
),  acc AS (
    SELECT
        COUNT(actor_login) OVER (ORDER BY first_seen_at) AS cnt,
        first_seen_at AS event_year
    FROM
        bots_with_first_seen AS bwfs
    ORDER BY event_year
)
SELECT ANY_VALUE(cnt) AS bots_total, event_year
FROM acc
GROUP BY event_year
ORDER BY event_year;

#### Example 4

In [ ]:
TODO: 